# Summa control notebook

In [1]:
# netcdf/numpy/xray/stats
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
import copy

# OS interaction
import sys, pickle, os

# import plotting
import seaborn as sns
import matplotlib
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.basemap import Basemap

import summapy

# Customize
sns.set_style("whitegrid")
sns.set_context('paper')
%matplotlib inline

In [2]:
# --------------------------------------------------------------------------------------------------------------------
# Directory Lists
# Unix
if 'linux' in sys.platform:
    dir_pre = '/home/lapok/gdrive/'
# Mac
elif 'darwin' in sys.platform:
    dir_pre = '/Users/karllapo/gdrive/'

# Project specific directories
dirProj = dir_pre + 'SnowHydrology/proj/ModTsfc/'
dirPrint = dirProj + 'Graphics'
dirData = dirProj + 'data'

# Summa specific directories/names
dirSumma = dirProj + 'summa/'
projName = 'summaTestCases'

In [3]:
# Site wide arguments
os.chdir(dirData)
dat = xr.open_dataset('SWA.ModTsfc.ModelForcing.nc')
datName = 'SWA.ModTsfc.ModelForcing.nc'
attr = xr.open_dataset('SWA.ModTsfc.Attributes.nc')

siteName = 'SWA'
myDecisions = {'astability': 'louisinv',
               'subRouting': 'qInstant',
               'alb_method': 'conDecay'}
allParams = {'albedoMinWinter': 0.6,
             'albedoMinSpring': 0.25,
             'albedoDecayRate': .5*10**6}

dirModel = '/Users/karllapo/gdrive/SnowHydrology/proj/ModTsfc/summa/ModTsfc_summa/'

# ----------
# Experiment names and identifiers
experiments = ['layers', 'thermalCond', 'stability', 'longwaveBias']

# Layer
expNames = {'layers': ['layerThin', 'layerThick', 'jrdn1991', 'CLM_2010']}
# Longwave bias
# expNames['longwaveBias'] = ['lowLW', 'obsLW', 'highLW']
# Thermal conductivity
arrayNames = ['tyen1965', 'melr1977', 'jrdn1991', 'smnv2000']
arrayAppend = ['Mid', 'Low', 'High']
expNames['thermalCond'] = [name + '_' + append for name in arrayNames for append in arrayAppend]
# Stability
arrayAppend = ['Low', 'Mid', 'High']
arrayNames = ['standard', 'louisinv', 'mahrtexp']
expNames['stability'] = [name + '_' + append for name in arrayNames for append in arrayAppend]


# Summa control function

In [4]:
def summaControlFunc(dirModel, siteName, expName, expID, attrFile, datFile, datFileName, allParams, allDecisions):
    summapy.summaFileManager.writeDefault(dirModel, siteName, expName);
    
    summapy.summaFileManager.localAttributesFile(attrFile, dirModel, siteName, expName);

    summapy.summaFileManager.paramTrial(list(allParams.keys()), list(allParams.values()),
                                        dirModel, siteName, expName, expID);

    summapy.summaFileManager.forcingFile(dirModel, siteName, expName, datFileName,
                                         np.array(1001), xrForce=datFile, expID=expID)   ; 
    
    summapy.summaFileManager.decision(allDecisions, dirModel, siteName, expName,
                                      pd.to_datetime(datFile.time[0].values),
                                      pd.to_datetime(datFile.time[-1].values), expID=expID);
    
    summapy.summaFileManager.fileManager(dirModel, '/ModTsfc_summa/', siteName, expName, expID=expID);

In [5]:
def summaRunCommand(expNames, exp, site, fName=None):
    if fName == None:
        fName = 'run' + site + exp + '.sh'
    with open(fName, "w+") as f:
    
        f.write('''#!/bin/bash

# Used to run the test cases for SUMMA

# There are two classes of test cases:
#  1) Test cases based on synthetic/lab data; and
#  2) Test cases based on field data.

# The commands assume that you are in the directory {localInstallation}//summaTestCases/settings/
# and that the control files are in {localInstallation}//summaTestCases/settings/

# Set the SUMMA executable (e.g. /usr/local/bin/summa.exe or wherever you have installed SUMMA)

BASEDIR=`pwd`
SUMMA_EXE="docker run -v ${BASEDIR}:/ModTsfc_summa bartnijssen/summa:docker"


# *************************************************************************************************
''')
        for expID in expNames[exp]:
            fileManagerName = 'summa_fileManager_' + expID + '.txt'
            f.write('${SUMMA_EXE} -p never -s _' + expID
                    +' -m /ModTsfc_summa/settings/'
                    + exp + '/' + site + '/' + fileManagerName)
            f.write('\n')


# Layer Experiments

In [6]:
# Set up modeling decisions. Non-specified decision will follow the default values set by summa and summapy

# UEB-like set-up
layerThin = {'zminLayer3': 100.0000,
             'zminLayer4': 100.0000,
             'zminLayer5': 100.0000,
             'zmaxLayer2_lower': 1000.0000,
             'zmaxLayer3_lower': 1000.0000,
             'zmaxLayer4_lower': 1000.0000,
             'zmaxLayer2_upper': 1000.0000,
             'zmaxLayer3_upper': 1000.0000,
             'zmaxLayer4_upper': 1000.0000}
layerThin.update(allParams)

# VIC-like set-up
layerThick = {'zminLayer1': 0.0500,
              'zminLayer2': 0.0200,
              'zminLayer3': 100.0000,
              'zminLayer4': 100.0000,
              'zminLayer5': 100.0000,
              'zmaxLayer1_lower': 0.2000,
              'zmaxLayer2_lower': 1000.0000,
              'zmaxLayer3_lower': 1000.0000,
              'zmaxLayer4_lower': 1000.0000,
              'zmaxLayer1_upper': 0.1500,
              'zmaxLayer2_upper': 1000.0000,
              'zmaxLayer3_upper': 1000.0000,
              'zmaxLayer4_upper': 1000.0000}
layerThick.update(allParams)

In [7]:
# Experiment specific files
expN = 'layers'
for expID in expNames[expN]:
    print(expID)
    # Parameterize for 2-layer thin, 2-layer thick, jordan, and clm
    if expID == 'layerThin':
        myDecisions['snowLayers'] = 'CLM_2010'
        summaControlFunc(dirModel, siteName, expN, expID, attr, dat, datName, layerThin, myDecisions)
    elif expID == 'layerThick':
        myDecisions['snowLayers'] = 'CLM_2010'
        summaControlFunc(dirModel, siteName, expN, expID, attr, dat, datName, layerThick, myDecisions)
    else:
        myDecisions['snowLayers'] = expID
        # Send to the control function
        summaControlFunc(dirModel, siteName, expN, expID, attr, dat, datName, allParams, myDecisions)

# Write the shell script for running this experiment
os.chdir(dirModel)
summaRunCommand(expNames, expN, siteName)

layerThin
1000.0    100.0    1000.0    0.6    100.0    1000.0    1000.0    1000.0    100.0    1000.0    500000.0    0.25
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
layerThick
1000.0    0.05    100.0    0.15    1000.0    0.6    0.2    100.0    1000.0    1000.0    0.02    500000.0    100.0    1000.0    1000.0    0.25
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
jrdn1991
500000.0    0.25    0.6
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
CLM_2010
500000.0    0.25    0.6
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager


# Thermal Conductivity Tests

In [8]:
# Set up modeling decisions. Non-specified decision will follow the default values set by summa and summapy
thermalCondVar = {'Mid': 0.3500, 'Low': 0.1000, 'High': 1.0000}

# Experiment specific files
expN = 'thermalCond'
thCondParam = {}
for expID in expNames[expN]:
    print(expID)
    paramSelection = expID.split('_')[1]
    decisionSelection = expID.split('_')[0]
    myDecisions['thCondSnow'] = decisionSelection
    thCondParam['fixedThermalCond_snow'] = thermalCondVar[paramSelection]
    thCondParam.update(allParams)

    summaControlFunc(dirModel, siteName, expN, expID, attr, dat, datName, thCondParam, myDecisions)

# Write the shell script for running this experiment
os.chdir(dirModel)
summaRunCommand(expNames, expN, siteName)

tyen1965_Mid
0.35    500000.0    0.25    0.6
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
tyen1965_Low
0.6    0.1    500000.0    0.25
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
tyen1965_High
0.6    1.0    500000.0    0.25
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
melr1977_Mid
0.6    0.35    500000.0    0.25
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
melr1977_Low
0.6    0.1    500000.0    0.25
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
melr1977_High
0.6    1.0    500000.0    0.25
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
jrdn1991_Mid
0.6    0.35    500000.0

## Turbulence

In [9]:
# Set up modeling decisions. Non-specified decision will follow the default values set by summa and summapy
expN = 'stability'
aerodynamic_z0 = {'Low': .001, 'Mid': .01, 'High': .1}
varName = ['z0Snow']
aStabilityParam = {}
for expID in expNames[expN]:
    print(expID)
    paramSelection = expID.split('_')[1]
    decisionSelection = expID.split('_')[0]
    myDecisions['astability'] = decisionSelection
    aStabilityParam['z0Snow'] = aerodynamic_z0[paramSelection]
    aStabilityParam.update(allParams)
    summaControlFunc(dirModel, siteName, expN, expID, attr, dat, datName, aStabilityParam, myDecisions)

# Write the shell script for running this experiment
os.chdir(dirModel)
summaRunCommand(expNames, expN, siteName)

standard_Low
0.6    500000.0    0.001    0.25
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
standard_Mid
0.6    500000.0    0.01    0.25
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
standard_High
0.6    500000.0    0.1    0.25
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
louisinv_Low
0.6    500000.0    0.001    0.25
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
louisinv_Mid
0.6    500000.0    0.01    0.25
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
louisinv_High
0.6    500000.0    0.1    0.25
Finished creating new summa_zParamTrial file
Finished creating new Decision file
Finished creating new file Manager
mahrtexp_Low
0.6    500000.0    